In [ ]:
!pip install tensorflow-addons
!pip install pyyaml h5py
!pip install toolz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.9.2


## Setup

In [ ]:
import os
# os.environ["MODIN_CPUS"] = "8"
# from distributed import Client
# client = Client()
import numpy as np
import math
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
# import allel
from scipy.spatial.distance import squareform
%matplotlib inline   
from toolz import interleave
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.9.2


## Hardware Config

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on TPU  grpc://10.121.2.74:8470
N_REPLICAS: 8


## Prepare the data

In [ ]:
# load genotype
genotypes = pd.read_csv('[path]/genotype_full.txt', sep='\t', index_col=0)
genotypes[genotypes == -1] = 0

headers = genotypes.columns[:]

In [ ]:
genotypes.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_02,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_03,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = genotypes
X.shape

(4390, 28220)

In [ ]:
# hyperparameters
feature_size = X.shape[1]
inChannel = 3
learning_rate = 0.01
weight_decay = 0.00001
embed_dim = 64  # Embedding size for each token
num_heads = 8 # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
regularization_coef_l1 = 1e-4
batch_size = 20
dropout_rate = 0.25
attention_range = 100
chunk_size = 1000

## Convert to tensorflow dataset

In [ ]:
@tf.function()
def add_attention_mask(X_sample, y_sample):
  depth = 3
  mask_size = tf.cast(X_sample.shape[0]*0.5, dtype=tf.int64)
  mask_idx = tf.reshape(tf.random.shuffle(tf.range(X_sample.shape[0]))[:mask_size], (-1, 1))
  updates = tf.math.add(tf.ones(shape=(mask_idx.shape[0]), dtype=tf.int64), 1)
  X_masked = tf.tensor_scatter_nd_update(X_sample, mask_idx, updates)

  return tf.one_hot(X_masked, depth), tf.one_hot(y_sample, depth-1)

In [ ]:
def map_values_1(x):
  return 0 if (x == 0 or x == 1) else 1

def map_values_2(x):
  return 0 if (x == 0 or x == 2) else 1

def get_dataset(x, chunk_start, chunk_end, start_offset, end_offset, batch_size, training=True):
  AUTO = tf.data.AUTOTUNE

  new_chunk_end = x.shape[1]

  dataset = tf.data.Dataset.from_tensor_slices((x,
                                                x[:, start_offset:new_chunk_end-end_offset]))
  # # Add Attention Mask
  
  if training:
    dataset = dataset.shuffle(x.shape[0], reshuffle_each_iteration=True)
    dataset = dataset.repeat()
  
  # Add Attention Mask
  dataset = dataset.map(add_attention_mask, num_parallel_calls=AUTO, deterministic=False)

  # Prefetech to not map the whole dataset
  dataset = dataset.prefetch(AUTO)

  dataset = dataset.batch(batch_size, drop_remainder=True, num_parallel_calls=AUTO)

  return dataset

## Custom Layers


In [ ]:
class CrossAttentionLayer(layers.Layer):
  def __init__(self, local_dim, global_dim,
               start_offset=0, end_offset=0,
               activation=tf.nn.gelu, dropout_rate=0.1,
               n_heads=num_heads):
    super(CrossAttentionLayer, self).__init__()
    self.local_dim = local_dim
    self.global_dim = global_dim
    self.dropout_rate = dropout_rate
    self.activation = activation
    self.start_offset = start_offset
    self.end_offset = end_offset
    self.num_heads = n_heads
    self.layer_norm00 = layers.LayerNormalization()
    self.layer_norm01 = layers.LayerNormalization()
    self.layer_norm1 = layers.LayerNormalization()
    self.ffn = tf.keras.Sequential(
          [
            layers.Dense(self.local_dim//2, activation=self.activation, 
                        ),
            layers.Dense(self.local_dim, 
                        activation=self.activation,
                        ), ]
      )
    self.add0 = layers.Add()
    self.add1 = layers.Add()
    self.attention = layers.MultiHeadAttention(num_heads=self.num_heads,
                                               key_dim=self.local_dim,)

  def call(self, inputs, training):
    local_repr = self.layer_norm00(inputs[0])
    global_repr = self.layer_norm01(inputs[1])
    query = local_repr[:, self.start_offset:local_repr.shape[1]-self.end_offset, :]
    key = global_repr
    value = global_repr

    # Generate cross-attention outputs: [batch_size, latent_dim, projection_dim].
    attention_output = self.attention(
        query, key, value
    )
    # Skip connection 1.
    attention_output = self.add0([attention_output, query])

    # Apply layer norm.
    attention_output = self.layer_norm1(attention_output)
    # Apply Feedforward network.
    outputs = self.ffn(attention_output)
    # Skip connection 2.
    outputs = self.add1([outputs, attention_output])
    return outputs

class MaskedTransformerBlock(layers.Layer):
  def __init__(self, embed_dim, num_heads, ff_dim, attention_range, start_offset=0, end_offset=0, attn_block_repeats=1, activation=tf.nn.gelu, dropout_rate=0.1, use_ffn=True):
    super(MaskedTransformerBlock, self).__init__()
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    self.ff_dim = ff_dim
    self.start_offset = start_offset
    self.end_offset = end_offset
    self.attention_range = attention_range
    self.attn_block_repeats = attn_block_repeats
    self.activation = activation
    self.dropout_rate = dropout_rate
    self.use_ffn = use_ffn
    self.att0 = [layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.embed_dim) for _ in range(attn_block_repeats)]
    if self.use_ffn:
      self.ffn = [tf.keras.Sequential(
          [
            layers.Dense(self.ff_dim, activation=self.activation, 
                        ),
            layers.Dense(self.embed_dim, 
                        activation=self.activation,
                        ), ]
      ) for _ in range(attn_block_repeats)]
    self.layer_norm0 = [layers.LayerNormalization() for _ in range(attn_block_repeats)]
    self.layer_norm1 = [layers.LayerNormalization() for _ in range(attn_block_repeats)]

  def build(self, input_shape):
    assert(self.end_offset >= 0)
    self.feature_size = input_shape[1]
    attention_mask = np.zeros((self.feature_size,
                               self.feature_size), dtype=bool)
    for i in range(self.start_offset, self.feature_size - self.end_offset):
      attention_indices = np.arange(max(0, i-self.attention_range), min(self.feature_size, i+self.attention_range))
      attention_mask[i, attention_indices] = True
    self.attention_mask = tf.constant(attention_mask[self.start_offset:self.feature_size-self.end_offset])

  def call(self, inputs, training):
    
    x = inputs
    for i in range(self.attn_block_repeats-1):
      x = self.layer_norm0[i](x)
      attn_output = self.att0[i](x, x)
      out1 = x + attn_output
      out1 = self.layer_norm1[i](out1)
      if self.use_ffn:
        ffn_output = self.ffn[i](out1)
        x = out1 + ffn_output
      else:
        x = out1

    x = self.layer_norm0[-1](inputs)
    attn_output = self.att0[-1](x[:, self.start_offset:x.shape[1]-self.end_offset, :], x,
                            )
    out1 = x[:, self.start_offset:x.shape[1]-self.end_offset, :] + attn_output
    out1 = self.layer_norm1[-1](out1)
    if self.use_ffn:
      ffn_output = self.ffn[-1](out1)
      x = out1 + ffn_output
    else:
      x = out1
    return x

class GenoEmbeddings(layers.Layer):
  def __init__(self, embedding_dim, 
               embeddings_initializer='glorot_uniform',
               embeddings_regularizer=None,
               activity_regularizer=None,
               embeddings_constraint=None):
    super(GenoEmbeddings, self).__init__()
    self.embedding_dim = embedding_dim
    self.embeddings_initializer = initializers.get(embeddings_initializer)
    self.embeddings_regularizer = regularizers.get(embeddings_regularizer)
    self.activity_regularizer = regularizers.get(activity_regularizer)
    self.embeddings_constraint = constraints.get(embeddings_constraint)

  def build(self, input_shape):
    # print(input_shape)
    
    self.num_of_allels = input_shape[-1]
    self.n_snps = input_shape[-2]
    self.position_embedding = layers.Embedding(
            input_dim=self.n_snps, output_dim=self.embedding_dim
        )
    self.embedding = self.add_weight(
            shape=(self.num_of_allels, self.embedding_dim),
            initializer=self.embeddings_initializer,
            trainable=True, name='geno_embeddings',
            regularizer=self.embeddings_regularizer,
            constraint=self.embeddings_constraint,
            experimental_autocast=False
        )
    self.positions = tf.range(start=0, limit=self.n_snps, delta=1)
  def call(self, inputs):
    self.immediate_result = tf.einsum('ijk,kl->ijl', inputs, self.embedding)
    return self.immediate_result + self.position_embedding(self.positions)


class Chunker(layers.Layer):
  def __init__(self, embed_dim, num_heads, ff_dim, chk_size=chunk_size,
               activation=tf.nn.gelu, dropout_rate=0.25, attn_block_repeats=1,
               attention_range=attention_range, include_embedding_layer=False):
    super(Chunker, self).__init__()
    self.concat = layers.Concatenate(axis=-2)
    self.chunk_size = chk_size
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    self.ff_dim = ff_dim
    self.activation = activation
    self.dropout_rate = dropout_rate
    self.attention_range = attention_range
    self.attn_block_repeats = attn_block_repeats
    self.include_embedding_layer = include_embedding_layer
    
  def build(self, input_shape):
    self.chunk_starts = list(range(0, input_shape[1], self.chunk_size))
    self.chunk_ends = []
    for cs in self.chunk_starts:
      self.chunk_ends.append(min(cs+self.chunk_size, input_shape[1]))
    self.mask_starts = [max(0, cs-self.attention_range) for cs in self.chunk_starts]
    self.mask_ends = [min(ce+self.attention_range, input_shape[1]) for ce in self.chunk_ends]
    self.chunkers = [SelfAttnChunk(self.embed_dim, self.num_heads, self.ff_dim,
                           attention_range,
                           include_embedding_layer=self.include_embedding_layer,
                           start_offset=cs - self.mask_starts[i], 
                            end_offset=self.mask_ends[i]-self.chunk_ends[i],
                           attn_block_repeats=self.attn_block_repeats) for i, cs in enumerate(self.chunk_starts)]

  def call(self, inputs, training):
    x = inputs
    chunks = [chunker(x[:, self.mask_starts[i]:self.mask_ends[i]]) for i, chunker in enumerate(self.chunkers)]
    y = self.concat(chunks)
    return y


class SelfAttnChunk(layers.Layer):
  def __init__(self, embed_dim, num_heads, ff_dim, attention_range,
               start_offset=0, end_offset=0,
               attn_block_repeats=1,
               include_embedding_layer=False):
    super(SelfAttnChunk, self).__init__()
    self.attention_range = attention_range
    self.ff_dim = ff_dim
    self.num_heads = num_heads
    self.embed_dim = embed_dim
    self.attn_block_repeats = attn_block_repeats
    self.include_embedding_layer = include_embedding_layer

    self.attention_block = MaskedTransformerBlock(self.embed_dim,
                                                   self.num_heads, self.ff_dim,
                                                   attention_range, start_offset,
                                                   end_offset, attn_block_repeats=1)
    if include_embedding_layer:
      self.embedding = GenoEmbeddings(embed_dim)
    

  def build(self, input_shape):
    pass
  
  def call(self, inputs, training):
    if self.include_embedding_layer:
      x = self.embedding(inputs)
    else:
      x = inputs
    x = self.attention_block(x)
    return x

class CrossAttnChunk(layers.Layer):
  def __init__(self, start_offset=0, end_offset=0, n_heads = num_heads):
    super(CrossAttnChunk, self).__init__()
    self.attention_range = attention_range
    self.start_offset = start_offset
    self.end_offset = end_offset
    self.n_heads = n_heads
    

  def build(self, input_shape):
    self.local_dim = input_shape[0][-1]
    self.global_dim = input_shape[1][-1]
    self.attention_block = CrossAttentionLayer(self.local_dim, self.global_dim,
                                              self.start_offset, self.end_offset,
                                              n_heads=self.n_heads)
    pass
  
  def call(self, inputs, training):
    x = inputs
    x = self.attention_block(x)
    return x


## Modules

In [ ]:
class ConvBlock(layers.Layer):
  def __init__(self, embed_dim):
    super(ConvBlock, self).__init__()
    self.embed_dim = embed_dim
    self.const = None
    self.conv000 = layers.Conv1D(embed_dim, 3, padding='same', activation=tf.nn.gelu,
                                 kernel_constraint=self.const,
                    )
    self.conv010 = layers.Conv1D(embed_dim, 5, padding='same', activation=tf.nn.gelu,
                                 kernel_constraint=self.const,
                    )
    self.conv011 = layers.Conv1D(embed_dim, 7, padding='same', activation=tf.nn.gelu,
                                 kernel_constraint=self.const,
                    )
    
    self.conv020 = layers.Conv1D(embed_dim, 7, padding='same', activation=tf.nn.gelu,
                                 kernel_constraint=self.const,
                    )
    self.conv021 = layers.Conv1D(embed_dim, 15, padding='same', activation=tf.nn.gelu,
                                 kernel_constraint=self.const,
                    )
    self.add = layers.Add()
    
    self.conv100 = layers.Conv1D(embed_dim, 3, padding='same',
                                 activation=tf.nn.gelu,
                                 kernel_constraint=self.const,)
    self.bn0 = layers.BatchNormalization()
    self.bn1 = layers.BatchNormalization()
    self.dw_conv = layers.DepthwiseConv1D(embed_dim, 1, padding='same')
    self.activation = layers.Activation(tf.nn.gelu)

  def call(self, inputs, training):
    # Could add skip connection here?
    xa = self.conv000(inputs)
    
    xb = self.conv010(xa)
    xb = self.conv011(xb)

    xc = self.conv020(xa)
    xc = self.conv021(xc)

    xa = self.add([xb, xc])
    xa = self.conv100(xa)
    xa = self.bn0(xa)
    xa = self.dw_conv(xa)
    xa = self.bn1(xa)
    xa = self.activation(xa)
    return xa

def chunk_module(embed_dim, num_heads, input_len, input_channels, attention_range,
               start_offset=0, end_offset=0,
               attn_block_repeats=1, include_embedding=False):
  projection_dim = embed_dim
  inputs = layers.Input(shape=(input_len, embed_dim))
  xa = inputs
  xa0 = SelfAttnChunk(projection_dim, num_heads, projection_dim//2, attention_range,
            start_offset, end_offset, 1, include_embedding_layer=False)(xa)
  
  xa = ConvBlock(projection_dim)(xa0)
  xa_skip = ConvBlock(projection_dim)(xa)

  xa = layers.Dense(projection_dim, activation=tf.nn.gelu)(xa)
  xa = ConvBlock(projection_dim)(xa)
  xa = CrossAttnChunk(0, 0)([xa, xa0])
  xa = layers.Dropout(0.25)(xa)
  xa = ConvBlock(projection_dim)(xa)

  xa = layers.Concatenate(axis=-1)([xa_skip, xa])
  
  xa = layers.Conv1D(projection_dim//2, 5, padding='same', activation=tf.nn.gelu)(xa)
  
  xa_out1 = layers.Conv1D(inChannel - 1, 5, padding='same', activation=tf.nn.softmax)(xa)
  
  model = keras.Model(inputs=inputs, outputs=xa_out1)
  return model

## Model

In [ ]:
class SplitTransformer(keras.Model):
  def __init__(
      self,
      embed_dim,
      num_heads,
      # latent_dim = 512,
      chunk_size=chunk_size,
      activation=tf.nn.gelu,
      dropout_rate=0.25,
      attn_block_repeats=1,
      attention_range=attention_range):
    super(SplitTransformer, self).__init__()
    self.embed_dim = embed_dim
    self.num_heads = num_heads
    self.chunk_size = chunk_size
    self.activation = activation
    self.dropout_rate = dropout_rate
    self.attn_block_repeats = attn_block_repeats
    self.attention_range = attention_range
    # self.latent_dim = latent_dim

  def build(self, input_shape):
    self.chunk_starts = list(range(0, input_shape[1], self.chunk_size))
    self.chunk_ends = []
    for cs in self.chunk_starts:
      self.chunk_ends.append(min(cs+self.chunk_size, input_shape[1]))
    self.mask_starts = [max(0, cs-self.attention_range) for cs in self.chunk_starts]
    self.mask_ends = [min(ce+self.attention_range, input_shape[1]) for ce in self.chunk_ends]
    self.chunkers = [chunk_module(self.embed_dim, self.num_heads,
                                  self.mask_ends[i] - self.mask_starts[i],
                                  inChannel, self.attention_range,
                                  start_offset=cs - self.mask_starts[i],
                                  end_offset=self.mask_ends[i]-self.chunk_ends[i],
                                  attn_block_repeats=1, include_embedding=True) for i,cs in enumerate(self.chunk_starts)]

    self.concat_layer = layers.Concatenate(axis=-2)
    self.embedding = GenoEmbeddings(self.embed_dim)
    super(SplitTransformer, self).build(input_shape)


  def call(self, inputs):
    x = self.embedding(inputs)
    chunks = [self.chunkers[i](x[:, self.mask_starts[i]:self.mask_ends[i]]) for i, chunker in enumerate(self.chunkers)]
    y = self.concat_layer(chunks)
    return y


In [ ]:
class MyCustomLoss(tf.keras.losses.Loss):

  def call(self, y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    
    loss_obj = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
    cat_loss = loss_obj(y_true, y_pred)

    loss_obj = tf.keras.losses.KLDivergence(reduction=tf.keras.losses.Reduction.SUM)
    kl_loss = loss_obj(y_true, y_pred)

    return cat_loss + kl_loss

In [ ]:
def create_model():
  model =  SplitTransformer(embed_dim=128,
      num_heads=4,
      attn_block_repeats=1,
      chunk_size=2000,
      activation="gelu")
  optimizer = tfa.optimizers.LAMB(learning_rate=learning_rate)
  model.compile(optimizer, loss=MyCustomLoss(), metrics=tf.keras.metrics.CategoricalAccuracy())
  return model


In [ ]:
model = create_model()
model.build((1, feature_size, inChannel))
model.summary()

Model: "split_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 2000, 2)           3619522   
                                                                 
 model_1 (Functional)        (None, 2000, 2)           3619522   
                                                                 
 model_2 (Functional)        (None, 2000, 2)           3619522   
                                                                 
 model_3 (Functional)        (None, 2000, 2)           3619522   
                                                                 
 model_4 (Functional)        (None, 2000, 2)           3619522   
                                                                 
 model_5 (Functional)        (None, 2000, 2)           3619522   
                                                                 
 model_6 (Functional)        (None, 2000, 2)     

In [ ]:
METRIC = "val_categorical_accuracy"

def create_callbacks(kfold=0, metric = METRIC):
    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='auto',
        factor=0.5,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='auto',
        patience= 10, 
        verbose=1,
        restore_best_weights=True
    )
    
    callbacks = [
                 reducelr,
                 earlystop
                 ]         
    
    return callbacks

## Training

In [ ]:
# A TPU V3-8 has 8 computing cores, the global batch size will be 1/16 x 8 = 8/128
BATCH_SIZE_BASE = 4
# Training configuration
BATCH_SIZE = BATCH_SIZE_BASE * N_REPLICAS if TPU else 32
BATCH_SIZE

32

In [ ]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold < 3:
    continue
  accuracies = []
  print(f"Training using fold {fold}")
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = f"[path]/YEAST/STI/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
    predict_onehot = model.predict(test_X_missing, verbose=1)
      # predict_onehot = model.predict([test_X_missing[i:i + 1, :, :], mMask])
      # only care the missing position
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      predict_missing_onehot = predict_onehot[i:i+1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot[i:i+1, :, :], axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

Training using fold 3
Epoch 1/1000
82/82 [==============================] - 909s 2s/step - loss: 6472.6919 - categorical_accuracy: 0.9876 - val_loss: 271592.1562 - val_categorical_accuracy: 0.8853 - lr: 0.0100
Epoch 2/1000
82/82 [==============================] - 91s 1s/step - loss: 1297.2960 - categorical_accuracy: 0.9979 - val_loss: 71834.9688 - val_categorical_accuracy: 0.9625 - lr: 0.0100
Epoch 3/1000
82/82 [==============================] - 86s 1s/step - loss: 1095.2350 - categorical_accuracy: 0.9982 - val_loss: 101658.2266 - val_categorical_accuracy: 0.9250 - lr: 0.0100
Epoch 4/1000
82/82 [==============================] - 91s 1s/step - loss: 987.7424 - categorical_accuracy: 0.9984 - val_loss: 19981.8906 - val_categorical_accuracy: 0.9858 - lr: 0.0100
Epoch 5/1000
82/82 [==============================] - 91s 1s/step - loss: 980.7889 - categorical_accuracy: 0.9984 - val_loss: 11467.9941 - val_categorical_accuracy: 0.9911 - lr: 0.0100
Epoch 6/1000
82/82 [===========================

100%|██████████| 1463/1463 [00:00<00:00, 12413.70it/s]


46/46 [==============================] - 123s 2s/step


100%|██████████| 1463/1463 [00:00<00:00, 2011.08it/s]


The average imputation accuracyon test data with 0.01 missing genotypes is 0.9987: 
Sensitivity: 0.9986695605082361
Specificity: 0.9986695605082361
F1-score macro: 0.9986692748495145
F1-score micro: 0.9986693038204796


...

The average imputation accuracyon test data with 0.1 missing genotypes is 0.9987: 
Sensitivity: 0.9986624447280104
Specificity: 0.9986624447280104
F1-score macro: 0.9986624814024088
F1-score micro: 0.9986624960700831


100%|██████████| 1463/1463 [00:00<00:00, 2870.46it/s]


46/46 [==============================] - 13s 273ms/step


100%|██████████| 1463/1463 [00:01<00:00, 863.88it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9986: 
Sensitivity: 0.9985844744049692
Specificity: 0.9985844744049692
F1-score macro: 0.9985844899410551
F1-score micro: 0.998584503265767


In [ ]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold < 2:
    continue
  accuracies = []
  print(f"Training using fold {fold}")
  # train_dataset, valid_dataset, test_dataset, steps_per_epoch, validation_steps = get_three_sets(X, attention_mask, Y_train, batch_size, population_label, missing_perc)
  # x_train, y_train, test_dataset, test_indices = get_three_sets(X, Y_train, random_state, population_label, missing_perc)
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  # kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=random_state)
  # for fold,(x_train_ids,x_valid_ids) in enumerate(kfold.split(x_train,y_train)):
  # print(f"Fold: {fold}")
  # print(x_train.shape)
  
  # print(x_train_ids.shape, x_valid_ids.shape)
  # x_train_data = x_train[x_train_ids]
  # x_valid_data = x_train[x_valid_ids]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = f"./drive/MyDrive/ShiLab/YEAST/STI/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
    predict_onehot = model.predict(test_X_missing, verbose=1)
      # predict_onehot = model.predict([test_X_missing[i:i + 1, :, :], mMask])
      # only care the missing position
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      predict_missing_onehot = predict_onehot[i:i+1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot[i:i+1, :, :], axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

Training using fold 2
Epoch 1/1000
82/82 [==============================] - 928s 2s/step - loss: 6747.5195 - categorical_accuracy: 0.9873 - val_loss: 313816.8750 - val_categorical_accuracy: 0.8870 - lr: 0.0100
Epoch 2/1000
82/82 [==============================] - 91s 1s/step - loss: 1368.5652 - categorical_accuracy: 0.9978 - val_loss: 17445.0859 - val_categorical_accuracy: 0.9913 - lr: 0.0100
Epoch 3/1000
82/82 [==============================] - 86s 1s/step - loss: 1069.3760 - categorical_accuracy: 0.9983 - val_loss: 224348.6562 - val_categorical_accuracy: 0.9226 - lr: 0.0100
Epoch 4/1000
82/82 [==============================] - 86s 1s/step - loss: 1029.1028 - categorical_accuracy: 0.9983 - val_loss: 154383.4375 - val_categorical_accuracy: 0.9510 - lr: 0.0100
Epoch 5/1000
82/82 [==============================] - 86s 1s/step - loss: 981.0383 - categorical_accuracy: 0.9985 - val_loss: 119279.9609 - val_categorical_accuracy: 0.9634 - lr: 0.0100
Epoch 6/1000
82/82 [========================

100%|██████████| 1463/1463 [00:00<00:00, 12483.28it/s]


46/46 [==============================] - 123s 2s/step


100%|██████████| 1463/1463 [00:00<00:00, 1857.26it/s]


The average imputation accuracyon test data with 0.01 missing genotypes is 0.9987: 
Sensitivity: 0.9987250606648491
Specificity: 0.9987250606648491
F1-score macro: 0.9987250524577261
F1-score micro: 0.9987250524764523


...

100%|██████████| 1463/1463 [00:00<00:00, 2829.25it/s]


46/46 [==============================] - 13s 273ms/step


100%|██████████| 1463/1463 [00:01<00:00, 826.46it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9986: 
Sensitivity: 0.9985982133524229
Specificity: 0.9985982133524229
F1-score macro: 0.9985981860148959
F1-score micro: 0.998598188338574
Training using fold 3
Epoch 1/1000


KeyboardInterrupt: ignored

In [ ]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  # if fold < 2:
  #   continue
  accuracies = []
  print(f"Training using fold {fold}")
  # train_dataset, valid_dataset, test_dataset, steps_per_epoch, validation_steps = get_three_sets(X, attention_mask, Y_train, batch_size, population_label, missing_perc)
  # x_train, y_train, test_dataset, test_indices = get_three_sets(X, Y_train, random_state, population_label, missing_perc)
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  # kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=random_state)
  # for fold,(x_train_ids,x_valid_ids) in enumerate(kfold.split(x_train,y_train)):
  # print(f"Fold: {fold}")
  # print(x_train.shape)
  
  # print(x_train_ids.shape, x_valid_ids.shape)
  # x_train_data = x_train[x_train_ids]
  # x_valid_data = x_train[x_valid_ids]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = f"./drive/MyDrive/ShiLab/YEAST/STI/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
    predict_onehot = model.predict(test_X_missing, verbose=1)
      # predict_onehot = model.predict([test_X_missing[i:i + 1, :, :], mMask])
      # only care the missing position
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      predict_missing_onehot = predict_onehot[i:i+1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot[i:i+1, :, :], axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    

Training using fold 1
Epoch 1/1000
82/82 [==============================] - 993s 2s/step - loss: 6761.2256 - categorical_accuracy: 0.9877 - val_loss: 111287.8672 - val_categorical_accuracy: 0.9206 - lr: 0.0100
Epoch 2/1000
82/82 [==============================] - 92s 1s/step - loss: 1449.3319 - categorical_accuracy: 0.9977 - val_loss: 48141.0703 - val_categorical_accuracy: 0.9606 - lr: 0.0100
Epoch 3/1000
82/82 [==============================] - 86s 1s/step - loss: 1207.0846 - categorical_accuracy: 0.9981 - val_loss: 183368.1094 - val_categorical_accuracy: 0.9255 - lr: 0.0100
Epoch 4/1000
82/82 [==============================] - 86s 1s/step - loss: 1053.2629 - categorical_accuracy: 0.9983 - val_loss: 351281.8125 - val_categorical_accuracy: 0.8767 - lr: 0.0100
Epoch 5/1000
82/82 [==============================] - 86s 1s/step - loss: 944.8292 - categorical_accuracy: 0.9984 - val_loss: 165401.6875 - val_categorical_accuracy: 0.9402 - lr: 0.0100
Epoch 6/1000
82/82 [========================

100%|██████████| 1464/1464 [00:00<00:00, 7961.96it/s]


46/46 [==============================] - 121s 2s/step


100%|██████████| 1464/1464 [00:00<00:00, 1791.17it/s]


The average imputation accuracyon test data with 0.01 missing genotypes is 0.9987: 
Sensitivity: 0.9987426925027328
Specificity: 0.9987426925027328
F1-score macro: 0.9987428747288258
F1-score micro: 0.9987428787350308


...

100%|██████████| 1464/1464 [00:00<00:00, 1789.46it/s]


46/46 [==============================] - 13s 273ms/step


100%|██████████| 1464/1464 [00:01<00:00, 811.87it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9986: 
Sensitivity: 0.9985931589169728
Specificity: 0.9985931589169728
F1-score macro: 0.9985932139238928
F1-score micro: 0.998593215678529
Training using fold 2
Epoch 1/1000


ResourceExhaustedError: ignored